<a href="https://colab.research.google.com/github/Aku02/Awesome-pytorch-list/blob/master/20_epoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [ ]:
!pip install -q tensorflow==2.3.0 # Use 2.3.0 for built-in EfficientNet
!pip install -q git+https://github.com/keras-team/keras-tuner@master # Use github head for newly added TPU support
!pip install -q cloud-tpu-client # Needed for sync TPU version
!pip install -U tensorflow-gcs-config==2.3.0 # Needed for using private dataset

     |████████████████████████████████| 61kB 3.0MB/s 
Requirement already up-to-date: tensorflow-gcs-config==2.3.0 in /usr/local/lib/python3.6/dist-packages (2.3.0)


In [ ]:
!pip install -q efficientnet
import os
import re
import numpy as np
import pandas as pd
import random
import math
from sklearn import metrics
from sklearn.model_selection import train_test_split
import tensorflow as tf
import efficientnet.tfkeras as efn
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
!pip install gcsfs
from tqdm.notebook import tqdm as tqdm

     |████████████████████████████████| 51kB 2.8MB/s 
     |████████████████████████████████| 92kB 3.5MB/s 
     |████████████████████████████████| 1.2MB 10.7MB/s 
     |████████████████████████████████| 266kB 20.5MB/s 
     |████████████████████████████████| 153kB 20.1MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3161 sha256=3ed1e138f92b0674a8f3329987e6e58ad50b6c1c3e6fdb5e622e6f1472e5f358
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # Sync TPU version
    from cloud_tpu_client import Client
    c = Client()
    c.configure_tpu_version(tf.__version__, restart_type='ifNeeded')
    
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
    

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.64.229.234:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.64.229.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.64.229.234:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [ ]:
EPOCHS = 20
BATCH_SIZE = 4 * strategy.num_replicas_in_sync
IMAGE_SIZE = [384,384]
# Seed
SEED = 100
# Learning rate
LR = 0.0001
# Number of classes
NUMBER_OF_CLASSES = 81313

In [ ]:
# Seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

# Function to decode our images (normalize and reshape)
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    # Convert image to floats in [0, 1] range
    image = tf.cast(image, tf.float32) / 255.0
    return image

# This function parse our images and also get the target variable
def read_tfrecord(example):
    TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        "label": tf.io.FixedLenFeature([], tf.int64)
        }
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    image = decode_image(example['image'])
    image=tf.reverse(image, axis=[-1])
    target = tf.cast(example['label'], tf.int32)
    return image, target

# This function load our tf records and parse our data with the previous function
def load_dataset(filenames, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        # Disable order, increase speed
        ignore_order.experimental_deterministic = False 
        
    # Automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # Use data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order)
    # Returns a dataset of (image, label) pairs
    dataset = dataset.map(read_tfrecord, num_parallel_calls = AUTO) 
    return dataset

# This function output the data so that we can use arcface
def arcface_format(image, target):
    image=tf.image.resize(image,[384,384])
    image=tf.reverse(image, axis=[-1])
    return {'inp1': image, 'inp2': target}, target

# Training data pipeline
def get_training_dataset(filenames, ordered = False):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    # The training dataset must repeat for several epochs
    dataset = dataset.repeat() 
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    # Prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO)
    return dataset

# Validation data pipeline
def get_validation_dataset(filenames, ordered = True, prediction = False):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    # If we are in prediction mode, use bigger batch size for faster prediction
    if prediction:
        dataset = dataset.batch(BATCH_SIZE * 4)
    else:
        dataset = dataset.batch(BATCH_SIZE)
    # Prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset



# Function for a custom learning rate scheduler with warmup and decay
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.0000005 * BATCH_SIZE
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        epoch=5
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = False)
    return lr_callback

# Function to calculate global average precision score
def gap_vector(pred, conf, true, return_x = False):
    '''
    Compute Global Average Precision (aka micro AP), the metric for the
    Google Landmark Recognition competition. 
    This function takes predictions, labels and confidence scores as vectors.
    In both predictions and ground-truth, use None/np.nan for "no label".

    Args:
        pred: vector of integer-coded predictions
        conf: vector of probability or confidence scores for pred
        true: vector of integer-coded labels for ground truth
        return_x: also return the data frame used in the calculation

    Returns:
        GAP score
    '''
    x = pd.DataFrame({'pred': pred, 'conf': conf, 'true': true})
    x.sort_values('conf', ascending = False, inplace = True, na_position = 'last')
    x['correct'] = (x.true == x.pred).astype(int)
    x['prec_k'] = x.correct.cumsum() / (np.arange(len(x)) + 1)
    x['term'] = x.prec_k * x.correct
    gap = x.term.sum() / x.true.count()
    if return_x:
        return gap, x
    else:
        return gap

class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

def generalized_mean_pool_2d(X):
    gm_exp = 3.0
    pool = (tf.reduce_mean(tf.abs(X**(gm_exp)), 
                        axis = [1, 2], 
                        keepdims = False) + 1.e-7)**(1./gm_exp)
    return pool
# Function to build our model using fine tunning (efficientnet)
def batch_gap(y_t, y_p):
    pred_cat = tf.argmax(y_p, axis=-1)    
    y_t_cat = tf.argmax(y_t, axis=-1) * tf.cast(
        tf.reduce_sum(y_t, axis=-1), tf.int64)
    
    n_pred = tf.shape(pred_cat)[0]
    is_c = tf.cast(tf.equal(pred_cat, y_t_cat), tf.float32)

    GAP = tf.reduce_mean(
          tf.cumsum(is_c) * is_c / tf.cast(
              tf.range(1, n_pred + 1), 
              dtype=tf.float32))
    
    return GAP
def generalized_mean_pool_2d(X):
    gm_exp = 3.0
    pool = (tf.reduce_mean(tf.abs(X**(gm_exp)), 
                        axis = [1, 2], 
                        keepdims = False) + 1.e-7)**(1./gm_exp)
    return pool
def get_model():

    with strategy.scope():

        margin = ArcMarginProduct(
            n_classes = NUMBER_OF_CLASSES, 
            s = 64, 
            m = 0.05, 
            name='head/arc_margin', 
            dtype='float32'
            )

        inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        label = tf.keras.layers.Input(shape = (), name = 'inp2')
        x0 = efn.EfficientNetB5(weights = 'imagenet', include_top = False)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x0)
        x = tf.keras.layers.Dropout(0.3)(x)
        x = tf.keras.layers.Dense(512)(x)
        x = margin([x, label])
        
        output = tf.keras.layers.Softmax(dtype='float32')(x)

        model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])

        opt = tf.keras.optimizers.Adam(learning_rate = LR)

        model.compile(
            optimizer = opt,
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
            ) 
        
        return model


# Count the number of observations with the tabular csv
def count_data_items(filenames):
    records = [int(filename.split('-')[-1].split('.')[0]) for filename in filenames]
    return sum(records)
print("REPLICAS: ", strategy.num_replicas_in_sync)

# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_PATH_1 ='gs://kds-4c1ac9004efd00b8a8bcfa9b3dcc800dbdf8978d612363a73d982469' 
GCS_PATH_2 ='gs://kds-7850820fffb4d34978de60497a6b764639bd2c8ff0405571fcdb6582'
DICT_PATH = '/content/gdrive/My Drive/Kaggle/train_encoded.csv'

REPLICAS:  8


In [ ]:
# Configuration
EPOCHS = 20
BATCH_SIZE = 32 * strategy.num_replicas_in_sync
IMAGE_SIZE = [384, 384]
# Seed
SEED = 100
# Learning rate
LR = 0.0001
# Number of classes
NUMBER_OF_CLASSES = 81313

# Training filenames directory
FILENAMES = tf.io.gfile.glob(GCS_PATH_1 + '/train*.tfrec') + tf.io.gfile.glob(GCS_PATH_2 + '/train*.tfrec')
# Read csv file
df = pd.read_csv(DICT_PATH)
# Using 20% of the data to validate
TRAINING_FILENAMES, VALIDATION_FILENAMES = train_test_split(FILENAMES, test_size = 0.20, random_state = SEED)
training_groups = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in TRAINING_FILENAMES]
validation_groups = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in VALIDATION_FILENAMES]
n_trn_classes = df[df['group'].isin(training_groups)]['landmark_id_encode'].nunique()
n_val_classes = df[df['group'].isin(validation_groups)]['landmark_id_encode'].nunique()
print(f'The number of unique training classes is {n_trn_classes} of {NUMBER_OF_CLASSES} total classes')
print(f'The number of unique validation classes is {n_val_classes} of {NUMBER_OF_CLASSES} total classes')

The number of unique training classes is 80937 of 81313 total classes
The number of unique validation classes is 64024 of 81313 total classes


In [ ]:
# Seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

# Function to decode our images (normalize and reshape)
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    # Convert image to floats in [0, 1] range
    image = tf.cast(image, tf.float32) / 255.0
    # Explicit size needed for TPU
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

# This function parse our images and also get the target variable
def read_tfrecord(example):
    TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        "target": tf.io.FixedLenFeature([], tf.int64)
        }
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    image = decode_image(example['image'])
    target = tf.cast(example['target'], tf.int32)
    return image, target

# This function load our tf records and parse our data with the previous function
def load_dataset(filenames, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        # Disable order, increase speed
        ignore_order.experimental_deterministic = False 
        
    # Automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # Use data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order)
    # Returns a dataset of (image, label) pairs
    dataset = dataset.map(read_tfrecord, num_parallel_calls = AUTO) 
    return dataset

# This function output the data so that we can use arcface
def arcface_format(image, target):
    return {'inp1': image, 'inp2': target}, target

# Training data pipeline
def get_training_dataset(filenames, ordered = False):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    # The training dataset must repeat for several epochs
    dataset = dataset.repeat() 
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    # Prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO)
    return dataset

# Validation data pipeline
def get_validation_dataset(filenames, ordered = True, prediction = False):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    # If we are in prediction mode, use bigger batch size for faster prediction
    if prediction:
        dataset = dataset.batch(BATCH_SIZE * 4)
    else:
        dataset = dataset.batch(BATCH_SIZE)
    # Prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

# Count the number of observations with the tabular csv
def count_data_items(filenames):
    records = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    df = pd.read_csv(DICT_PATH)
    n = df[df['group'].isin(records)].shape[0]
    return n

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES  = count_data_items(VALIDATION_FILENAMES)
print(f'Training with {NUM_TRAINING_IMAGES} images')
print(f'Validating with {NUM_VALIDATION_IMAGES} images')

# Function for a custom learning rate scheduler with warmup and decay
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.0000005 * BATCH_SIZE
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = False)
    return lr_callback

# Function to calculate global average precision score
def gap_vector(pred, conf, true, return_x = False):
    '''
    Compute Global Average Precision (aka micro AP), the metric for the
    Google Landmark Recognition competition. 
    This function takes predictions, labels and confidence scores as vectors.
    In both predictions and ground-truth, use None/np.nan for "no label".

    Args:
        pred: vector of integer-coded predictions
        conf: vector of probability or confidence scores for pred
        true: vector of integer-coded labels for ground truth
        return_x: also return the data frame used in the calculation

    Returns:
        GAP score
    '''
    x = pd.DataFrame({'pred': pred, 'conf': conf, 'true': true})
    x.sort_values('conf', ascending = False, inplace = True, na_position = 'last')
    x['correct'] = (x.true == x.pred).astype(int)
    x['prec_k'] = x.correct.cumsum() / (np.arange(len(x)) + 1)
    x['term'] = x.prec_k * x.correct
    gap = x.term.sum() / x.true.count()
    if return_x:
        return gap, x
    else:
        return gap

class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output


# Function to build our model using fine tunning (efficientnet)
def get_model():

    with strategy.scope():

        margin = ArcMarginProduct(
            n_classes = NUMBER_OF_CLASSES, 
            s = 64, 
            m = 0.05, 
            name='head/arc_margin', 
            dtype='float32'
            )

        inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        label = tf.keras.layers.Input(shape = (), name = 'inp2')
        x0 = efn.EfficientNetB0(weights = 'imagenet', include_top = False)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x0)
        x = tf.keras.layers.Dropout(0.3)(x)
        x = tf.keras.layers.Dense(512)(x)
        x = margin([x, label])
        
        output = tf.keras.layers.Softmax(dtype='float32')(x)

        model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])

        opt = tf.keras.optimizers.Adam(learning_rate = LR)

        model.compile(
            optimizer = opt,
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
            ) 
        
        return model

# Seed everything
seed_everything(SEED)

# Build training and validation generators
train_dataset = get_training_dataset(TRAINING_FILENAMES, ordered = False)
val_dataset = get_validation_dataset(VALIDATION_FILENAMES, ordered = True, prediction = False)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

model = get_model()
# Using a checkpoint to save best model (want the entire model, not only the weights)
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'/content/gdrive/My Drive/Kaggle/noisy-student/baseline_model_effb0_arcface.h5', 
                                                 monitor = 'val_loss', 
                                                 save_best_only = True, 
                                                 save_weights_only = False)
# Using learning rate scheduler
cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', 
                                                       mode = 'min', 
                                                       factor = 0.5, 
                                                       patience = 1, 
                                                       verbose = 1, 
                                                       min_delta = 0.0001)

# Train and evaluate our model
history = model.fit(train_dataset,  
                    steps_per_epoch = STEPS_PER_EPOCH,
                    epochs = 2,
                    callbacks = [get_lr_callback(), checkpoint],
                    validation_data = val_dataset,
                    verbose = 1
                    )

# Restart tpu
tf.tpu.experimental.initialize_tpu_system(tpu)
# Load best model
model = tf.keras.models.load_model('/content/gdrive/My Drive/Kaggle/noisy-student/baseline_model_effb0_arcface.h5')

# Reset val dataset, now in prediction mode
val_dataset = get_validation_dataset(VALIDATION_FILENAMES, ordered = True, prediction = True)
# Get ground truth target for the fold
val_target = val_dataset.map(lambda image, target: target).unbatch()
val_targets = list(next(iter(val_target.batch(NUM_VALIDATION_IMAGES))).numpy())

 # Predictions
val_image = val_dataset.map(lambda image, target: image['inp1'])
# Transform validation dataset as a numpy iterator
val_image = val_image.as_numpy_iterator()
# Initiate empty list to store predictions and confidences
target_predictions = []
target_confidences = []
# Iterate over validation images and predict in batches of 1024 images
batches = math.ceil(NUM_VALIDATION_IMAGES / (BATCH_SIZE * 4))
for image in tqdm(val_image, total = batches):
    prediction = model.predict(image)
    target_prediction = np.argmax(prediction, axis = -1)
    target_confidence = np.max(prediction, axis = -1)
    target_predictions.extend(list(target_prediction))
    target_confidences.extend(list(target_confidence))

# Calculate global average precision for the fold
gap = gap_vector(target_predictions, target_confidences, val_targets)
accuracy_score = metrics.accuracy_score(val_targets, target_predictions)
print(f'Our global average precision score is {gap}')
print(f'Our accuracy score is {accuracy_score}')

Training with 1264378 images
Validating with 316092 images
16809984/16804768 [==============================] - 0s 0us/step
Epoch 1/2
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


   2/4938 [..............................] - ETA: 25:16 - loss: 18.6245 - sparse_categorical_accuracy: 0.0000e+00WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0083s vs `on_train_batch_end` time: 0.6028s). Check your callbacks.


4938/4938 [==============================] - 3463s 701ms/step - loss: 16.8251 - sparse_categorical_accuracy: 6.8031e-04 - val_loss: 15.0799 - val_sparse_categorical_accuracy: 0.0032
Epoch 2/2
 286/4938 [>.............................] - ETA: 47:16 - loss: 14.9464 - sparse_categorical_accuracy: 0.0029

In [ ]:
model.save_weights('/content/gdrive/My Drive/Kaggle/noisy-student/baseline_model_effb0_arcface.h5')

In [ ]:
# # Training filenames directory
# FILENAMES_1 = tf.io.gfile.glob(GCS_PATH_1 + '/train*.tfrec') 
# FILENAMES_2 = tf.io.gfile.glob(GCS_PATH_2 + '/train*.tfrec')
# FILENAMES = FILENAMES_1 + FILENAMES_2
# # Read csv file
# df = pd.read_csv('/content/gdrive/My Drive/Kaggle/train.csv')
# # Using 20% of the data to validate
# TRAINING_FILENAMES, VALIDATION_FILENAMES = train_test_split(FILENAMES, test_size = 0.20, random_state = SEED)
# TRAINING_FILENAMES=TRAINING_FILENAMES[9:]
# NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
# NUM_VALIDATION_IMAGES  = count_data_items(VALIDATION_FILENAMES)
# print(f'Training with {NUM_TRAINING_IMAGES} images')
# print(f'Validating with {NUM_VALIDATION_IMAGES} images')
# # Seed everything
# seed_everything(SEED)

In [ ]:
import cv2
# Build training and validation generators
train_dataset = get_training_dataset(TRAINING_FILENAMES, ordered = False)
val_dataset = get_validation_dataset(VALIDATION_FILENAMES, ordered = True, prediction = False)
STEPS_PER_EPOCH = n_trn_classes // BATCH_SIZE
model = get_model()
# Using a checkpoint to save best model (want the entire model, not only the weights)
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'/content/gdrive/My Drive/Kaggle/noisy-student/baseline_model_effb0_arcface.h5', 
                                                 monitor = 'val_loss', 
                                                 save_best_only = True, 
                                                 save_weights_only = False)
# Using learning rate scheduler
cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', 
                                                       mode = 'min', 
                                                       factor = 0.5, 
                                                       patience = 1, 
                                                       verbose = 1, 
                                                       min_delta = 0.0001)

# Train and evaluate our model
history = model.fit(train_dataset,  
                    steps_per_epoch = STEPS_PER_EPOCH,
                    epochs = 2,
                    callbacks = [get_lr_callback()],
                    validation_data = val_dataset,
                    verbose = 1
                    )

Epoch 1/2
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


InvalidArgumentError: ignored

In [ ]:
import cv2
# Build training and validation generators
train_dataset = get_training_dataset(TRAINING_FILENAMES, ordered = False)
val_dataset = get_validation_dataset(VALIDATION_FILENAMES, ordered = True, prediction = False)
STEPS_PER_EPOCH = n_trn_classes // BATCH_SIZE
model = get_model()
model.load_weights('/content/gdrive/My Drive/google_landmark/B5.hdf5')

115515392/115515256 [==============================] - 1s 0us/step


OSError: ignored

In [ ]:

# Using a checkpoint to save best model (want the entire model, not only the weights)
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'/content/gdrive/My Drive/Kaggle/noisy-student/baseline_model_effb0_arcface.h5', 
                                                 monitor = 'val_loss', 
                                                 save_best_only = True, 
                                                 save_weights_only = False)
# Using learning rate scheduler
cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', 
                                                       mode = 'min', 
                                                       factor = 0.5, 
                                                       patience = 1, 
                                                       verbose = 1, 
                                                       min_delta = 0.0001)

# Train and evaluate our model
history = model.fit(train_dataset,  
                    steps_per_epoch = STEPS_PER_EPOCH,
                    epochs = 1,
                    callbacks = [get_lr_callback()],
                    validation_data = val_dataset,
                    verbose = 1
                    )

NameError: ignored

In [ ]:
model.save_weights('/content/gdrive/My Drive/google_landmark/B5.hdf5')